本文介绍传统机器学习算法中的“支撑向量机（SVM）”，在深度学习变得流行之前，SVM是许多论文中经常会用到的算法。它有非常完备的数学推导，我在刚接触它的时候也被搞得云里雾里。现在打算系统的介绍一下它。

本文一共分成以下几个部分
1. 线性可分支撑向量机及其对偶问题
2. 线性不可分支撑向量机及其对偶问题
2. 非线性支撑向量机（核技巧）

**声明**：文章中用到的上下标，上标的标示是括号加数字，例如$\textbf{x}^{(i)}$表示样本集中第i个样本点，加粗的$\textbf{x}$表示这是一个向量。下标$x_i$表示向量$\textbf{x}$的第i个维度

## 线性可分支撑向量机及其对偶问题
这是一个最简单的SVM算法，通过这个例子，将会认识到利用SVM算法求解一个简单的二分类问题的整个流程。后面的线性不可分问题以及核技巧，都是在基于线性可分的基础上进行的拓展。所以这个环节会介绍的比较详细，我会尽量讲清楚公式推导的部分。

### 问题描述
给定一组训练数据集

\begin{equation}
T=\{(\textbf{x}^{(1)},y^{(1)}),(\textbf{x}^{(2)},y^{(2)}),\cdots, (\textbf{x}^{(N)}, y^{(N)})\}
\end{equation}

该数据集包含N个样本点。每个样本点的$\textbf{x}$加粗显示，表示这是一个向量，$\textbf{x}\in \mathbb{R}^{n}$，当然如果n=1，则$\textbf{x}$是一个标量。

在$\textbf{x}^{(i)}=(x^{(i)}_1,x^{(i)}_2,\cdots, x^{(i)}_n)$中的每一个维度，表示该样本点的一个特征，样本集中的每个样本点有n个维度或特征。

$y^{(i)}$表示第i个样本点的类别，$y\in\{+1, -1\}$，当$y^{(i)}=1$，则表示$x^{(i)}$是正例。当$y^{(i)}=-1$，则表示$x^{(i)}$是负例。学习的目标就是要找到一个超平面，这个超平面把空间分成两个部分，使得样本集中的正负样本点分别位于各自的部分。这个超平面用方程表示为$\textbf{w}\cdot \textbf{x}+b=0$，它由法相量$\textbf{w}$和截距项$b$所决定。

需要注意的是，对于一组线性可分的数据集，存在着无穷多个超平面可以把这组数据集的正负例完全分开。我们要找的是一个最优的超平面，以下将解释什么样的超平面算最优。

<img src="img/二分类问题样例.png">

### 最优超平面
上面介绍到，超平面由法相量$\textbf{w}$和截距项$b$所决定，那么这里定义最优超平面对应的法相量和截距项分别为$\textbf{w}^{\star}$和$b^{\star}$。则最优超平面的方程为$\textbf{w}^{\star}\cdot \textbf{x} + b^{\star}=0$

在上面的图7.1中，样本点的特征是二维的，所以利用一条直线可以将样本点分割开。用o表示正例，x表示负例。在正例中有三个点A、B、C，可以看到这三个点都被直线正确的分类了。但是仔细看这三个点到直线的距离则有所不同，C离直线的距离最近，A离得最远，B居中，这里说的距离可以理解为垂直于直线的距离。当距离越大时，表示分类的结果比较可信，当距离越小时，表示分类的结果比较不可信。为什么要说可信不可信，是因为我们获得的样本集T只是从众多样本点中随机抽样得到的，假如在运气不好的情况下，获得的样本集比较诡异，那么虽然分类直线在这个样本集上做到了完全分类正确，但是这条分类直线在其他不在样本集上的样本点，可能表现就会不佳。因为我们在抽样的时候没有做好，使得样本无法代表总体。而基于这样的样本集得到的分类直线，则不是一条好的分类直线。


在这个例子中，C点距离直线最近，表示虽然直线把C点的类别分对了，但是差一点就分错了。因为有上述抽样问题存在，我们不敢把C点刚刚好分对，而要尽可能的让它远离分类直线。通过使得这个距离最大化，得到最优的分类超平面。

为了解释最优超平面，首先要引入函数间隔和几何间隔的概念。

#### 函数间隔
超平面的方程式是$\textbf{w}\cdot \textbf{x} + b=0$，则样本点$(\textbf{x}^{(i)}, y^{(i)})$到超平面$(\textbf{w}, b)$的函数间隔为
\begin{equation}
\hat{\gamma}^{(i)}= y^{(i)}(\textbf{w}\cdot \textbf{x} + b)
\end{equation}
为什么可以这么定义，原因是对于超平面$(\textbf{w}, b)$而言，样本点$(\textbf{x}^{(i)}, y^{(i)})$到超平面的距离用$\textbf{w}\cdot \textbf{x}+b$表示，这是一个描述距离的值，它的绝对值表示距离平面的远近。这个值可正可负。如果样本点在平面的上方，即平面把它判成正例，则该值是正的，如果样本点在平面的下方，即平面把它判成负例，则该值是负的。

通过比较$\textbf{w}\cdot \textbf{x}+b$的符号与$y^{(i)}$的符号是否相同，可以判断平面是否正确分类。所以我们用$y^{(i)}(\textbf{w}\cdot \textbf{x} + b)$来表示分类正确和距离平面远近，也就是上文函数间隔的定义。

在样本集T中的所有样本点中找到$\hat{\gamma}^{(i)}$最小的一个，即为超平面$(\textbf{w}, b)$关于样本集T的函数间隔
\begin{equation}
\hat{\gamma} = \min_{i=1\cdots N}{\hat{\gamma}^{(i)}}
\end{equation}

#### 几何间隔
上文中的分离超平面方程式$(\textbf{w}, b)：\textbf{w} \cdot \textbf{x} + b=0$，如果在方程两边同时乘以一个系数c，则不改变分离超平面。但是函数间隔却会变成原来的c倍。为了保证样本点到超平面的距离不受到系数的影响，引入几何间隔的概念，即在计算距离的时候对超平面方程式中的参数做归一化。

$$
\begin{equation}
\begin{aligned}
\gamma^{(i)} &= y^{(i)}\frac{(\textbf{w}\cdot \textbf{x} + b)}{\|\textbf{w}\|} \\
 &=y^{(i)}\left(\frac{\textbf{w}}{\|\textbf{w}\|} \cdot \textbf{x} + \frac{b}{\|\textbf{w}\|}\right)
\end{aligned}
\end{equation}
$$

则分离超平面关于样本集T的几何间隔为

\begin{equation}
\gamma = \min_{i=1\cdots N}{\gamma}^{(i)}
\end{equation}

所以函数间隔和集合间隔的关系是

$$
\begin{equation}
\begin{aligned}
\gamma = \frac{\hat{\gamma}}{\|\textbf{w}\|} \\
\gamma^{(i)} = \frac{\hat{\gamma}^{(i)}}{\|\textbf{w}\|}
\end{aligned}
\end{equation}
$$

#### 间隔最大化
这里的间隔指几何间隔最大化。对于一个线性可分的数据集T，存在无穷多个完全分类正确的超平面，通过使得几何间隔$\gamma$最大化，找到最优的分离超平面$(\textbf{w}^{\star}, b^{\star})$

我们可以把上述过程用数学方式表达出来

$$
\begin{equation}
\begin{aligned}
\max_{\textbf{w}, b} \quad &\gamma \\
\text{s. t. } \quad &y^{(i)}\left(\frac{\textbf{w}}{\|\textbf{w}\|} \cdot \textbf{x} + \frac{b}{\|\textbf{w}\|}\right) \geq \gamma, \quad i=1,2,\cdots, N
\end{aligned}
\end{equation}
$$

根据函数间隔与几何间隔之间的关系，我们也可以把上式写成
$$
\begin{equation}
\begin{aligned}
\max_{\textbf{w}, b} \quad &\frac{\hat{\gamma}}{\|\textbf{w}\|} \\
\text{s. t. } \quad &y^{(i)}\left(\textbf{w} \cdot \textbf{x} + b\right) \geq \hat{\gamma}, \quad i=1,2,\cdots, N
\end{aligned}
\end{equation}
$$

上文提到如果用到函数间隔，要注意超平面的系数$\textbf{w}$和$b$按照系数c等比例增大或缩小的问题。假设将$\textbf{w}$和$b$等比例的变成$c\textbf{w}$和$cb$，则函数间隔变成了$c\hat{\gamma}$。带入用函数间隔表示的目标问题和约束条件中后发现没有产生任何影响。所以不失一般性，这里取函数间隔$\hat{\gamma}=1$

所以问题变成
$$
\begin{equation}
\begin{aligned}
\max_{\textbf{w}, b} \quad &\frac{1}{\|\textbf{w}\|} \\
\text{s. t. } \quad &y^{(i)}\left(\textbf{w} \cdot \textbf{x} + b\right) -1 \geq 0, \quad i=1,2,\cdots, N
\end{aligned}
\end{equation}
$$

为了后续推导的方便，利用最小化$\frac{1}{\|\textbf{w}\|}$和最大化$\frac{1}{2}\|\textbf{w}\|^2$是等价的，于是把问题修改成
$$
\begin{equation}
\begin{aligned}
\min_{\textbf{w}, b} \quad &\frac{1}{2}\|\textbf{w}\|^2 \\
\text{s. t. } \quad &y^{(i)}\left(\textbf{w} \cdot \textbf{x} + b\right) -1 \geq 0, \quad i=1,2,\cdots, N
\end{aligned}
\end{equation}
$$

这是一个目标函数为二次函数，约束条件为一次的凸二次规划问题。通过求解该问题得到$(\textbf{w}^{\star}, b^{\star})$

### 凸二次问题求解
#### 拉格朗日乘子法

对于上述问题，我们称之为原始问题。利用拉格朗日对偶性质，可以把问题转变成对偶问题。
定义拉格朗日函数
$$
\begin{equation}
L(\textbf{w},b,\alpha) = \frac{1}{2}\|\textbf{w}\|^2 - \sum_{i=1}^N{\alpha^{(i)}y^{(i)}(\textbf{w}\cdot \textbf{x}^{(i)}+b)} + \sum_{i=1}^N{\alpha^{(i)}}
\end{equation}
$$
其中N个$\alpha^{(i)},i=1\cdots N$组成拉格朗日乘子向量$\alpha=(\alpha^{(1)},\alpha^{(2)},\cdots,\alpha^{(N)})$，即样本点的个数与拉格朗日乘子个数相同。

通过拉格朗日乘子，把N个不等式约束条件，转变到目标函数中的部分。得到了拉格朗日函数$L$。


#### 拉格朗日极小极大问题

考虑以下优化问题
$$
\begin{equation}
\begin{aligned}
\max_{\textbf{x}\in \mathbb{R}^n}\quad &f(\textbf{x}) \\
\text{s.t } \quad &c^{(i)}(\textbf{x}) \leq 0 \quad i=1,2,\cdots, k \\
&h^{(j)}(\textbf{x}) = 0 \quad j=1,2,\cdots, l
\end{aligned}
\end{equation}
$$
这是一个标准的带约束的优化问题，称为原始问题(primal problem)。下面我们要利用拉格朗日对偶性质求解该问题。


假设$f(\textbf{x})$，$c(\textbf{x})$和$h(\textbf{x})$在$\mathbb{R}^n$上是连续可微的（函数存在导函数，且导函数是连续的）。引入拉格朗日函数

$$
\begin{equation}
L(\textbf{x},\alpha,\beta) = f(\textbf{x}) + \sum_{i=1}^k{\alpha^{(i)}c^{(i)}(\textbf{x})} + \sum_{j=1}^l{\beta^{(j)}h^{(j)}(\textbf{x})}
\end{equation}
$$

其中$\alpha^{(i)}$和$\beta^{(j)}$是拉格朗日乘子，且$\alpha^{(i)} \geq 0$。
拉格朗日函数$L(\textbf{x},\alpha,\beta)$有三个变量，分别是$\textbf{x}$，$\alpha$和$\beta$

定义关于$\textbf{x}$的函数
$$
\begin{equation}
\theta_{P}(\textbf{x}) = \max_{\alpha,\beta;\alpha^{(i)}\geq 0}L(\textbf{x},\alpha, \beta)
\end{equation}
$$
下表P表示primal原始问题的意思。可以看到，这个$\theta_P(\textbf{x})$函数从拉格朗日函数演变而来。它只有一个变量$\textbf{x}$，因为剩余的两个变量$\alpha$和$\beta$在求$\max_{\alpha,\beta;\alpha^{(i)}\geq 0}L(\textbf{x},\alpha, \beta)$过程中找到了能够使拉格朗日函数最大的$\alpha$和$\beta$。这里用$\alpha^{\star}$和$\beta^{\star}$表示。所以$\alpha$和$\beta$变量已经固定，即为常数。

分析$\theta_P(\textbf{x})$的取值范围。

**如果存在$\textbf{x}$使得在原始问题中的约束条件不满足**，即存在$c^{(i)}(\textbf{x}) > 0$或$h^{(i)}(\textbf{x}) \neq 0$，则可以通过调节$\alpha$和$\beta$使得$\theta_P(\textbf{x})$函数达到无穷大。例如$c^{(i)}(\textbf{x}) > 0$，则令对应的$\alpha^{(i)}\rightarrow +\infty$。如果$h^{(j)}(\textbf{x}) \neq 0$，则根据符号的方向决定$\beta^{(j)}$的方向，使得$\beta^{(j)}h^{(j)}(\textbf{x}) \rightarrow +\infty$，将其他拉格朗日乘子取0即可。

**如果所有的$\textbf{x}$都满足在原始问题中的约束条件**，则将所有的$\alpha^{(i)}$取0，可以使得$\theta_P(\textbf{x})$取到最大值，即$\theta_P(\textbf{x})=f(\textbf{x})$。

综合考虑两种情况得到$\theta_P{\textbf{x}}$的取值范围
$$ 
\begin{equation}
\theta_P(\textbf{x}) =\left\{
\begin{aligned}
& f(\textbf{x}) \quad \textbf{x}\text{满足原始问题约束} \\
& +\infty \quad \text{其他}
\end{aligned}
\right.
\end{equation}
$$

可以看到如果$\textbf{x}$满足原始问题的约束条件，则$\theta_P(\textbf{x})$可以取到$f(\textbf{x})$。原始问题是通过调节$\textbf{x}$对$f(\textbf{x})$进行最小化，则通过调节$\textbf{x}$来最小化$\theta_P(\textbf{x})$函数，这与最小化原始问题是等价的，即
$$
\begin{equation}
\min_{\textbf{x}}{\theta_{P}(\textbf{x})} = \min_{\textbf{x}}{\max_{\alpha,\beta;\alpha^{(i)}\geq 0}L(\textbf{x},\alpha, \beta)}
\end{equation}
$$

则原始问题转变成了拉格朗日函数的极小极大问题，定义$p^{\star}=\min_{\textbf{x}}\theta_P(\textbf{x})$为原始问题的最优值。

#### 拉格朗日对偶问题
在上文中提到的拉格朗日函数的极小极大问题，求解的步骤是
1. 通过调节拉格朗日乘子$\alpha^{(i)}$和$\beta^{(j)}$使得$L(\textbf{x},\alpha, \beta)$最大化。
2. 通过在样本集中选择$\textbf{x}$使得$\theta_P(\textbf{x})$函数最小。

现在我们将两个步骤的顺序交换位置
1. 先对$L(\textbf{x},\alpha, \beta)$，通过调节$\textbf{x}$实现最小化，定义
$$
\begin{equation}
\theta_{D}(\alpha,\beta) = \min_{\textbf{x}}L(\textbf{x},\alpha, \beta)
\end{equation}
$$

2. 再考虑调节$\alpha$和$\beta$去最大化$\theta_{D}(\alpha,\beta)$函数，即
$$
\begin{equation}
\max_{\alpha,\beta;\alpha^{(i)}\geq 0}{\theta_{D}(\alpha,\beta)} = \max_{\alpha,\beta;\alpha^{(i)}\geq 0}{\min_{\textbf{x}}L(\textbf{x},\alpha, \beta)}
\end{equation}
$$

这是拉格朗日的极大极小问题。如果把第2步中要求$\alpha^{(i)}\geq 0$表示成约束条件，则极大极小问题可以写成约束最优化问题，即
$$
\begin{equation}
 \max_{\alpha,\beta}{\theta_{D}(\alpha,\beta)} = \max_{\alpha,\beta}{\min_{\textbf{x}}L(\textbf{x},\alpha, \beta)} \\
 \text{s.t} \quad \alpha^{(i)}\geq 0 \quad i=1,2,\cdots,k
\end{equation}
$$

这种写法称为原始问题的对偶问题。定义$d^{\star}=\max_{\alpha,\beta;\alpha^{(i)}\geq 0}{\theta_{D}(\alpha,\beta)}$为对偶问题的最优值。

## 非线性可分支撑向量机及其对偶问题

## 核函数

## SMO序列最小化优化算法